In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - test_effsel_grid
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Do some simple tests of the effective selection function grid to try 
and determine what was going wrong with the fits
'''

__author__ = "James Lane"

In [ ]:
### Imports

## Basic
import numpy as np
import sys, os, pdb, dill as pickle, copy

## Matplotlib
from matplotlib import pyplot as plt

## Astropy
from astropy import units as apu

## Project-specific
sys.path.insert(0,'../../../../../src/')
from ges_mass import mass as pmass
from ges_mass import densprofiles as pdens
from ges_mass import util as putil

### Scale parameters
ro = 8.275
vo = 220
zo = 0.0208 # Bennett+ 2019

### Notebook setup

%matplotlib inline
plt.style.use('../../../../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Preliminaries

In [ ]:
# Pathing
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','NDMOD',
            'DMOD_MIN','DMOD_MAX']
data_dir_base,apogee_dr,apogee_results_vers,gaia_dr,ndmod,dmod_min,dmod_max \
    = putil.parse_config_dict(cdict,keywords)
data_dir = data_dir_base+'gaia_apogee/'
gaia_apogee_dir = 'apogee_'+apogee_dr+'_'+apogee_results_vers+'_gaia_'+gaia_dr+'/'

### Selection function and effective selection function grid

In [ ]:
# Filenames
apogee_SF_filename = data_dir+gaia_apogee_dir+'apogee_SF.dat'
apogee_effSF_filename = data_dir+gaia_apogee_dir+'apogee_effSF_grid_inclArea_z0.001_logAge10.0.dat'
apogee_effSF_mask_filename = data_dir+gaia_apogee_dir+'apogee_effSF_grid_mask_z0.001_logAge10.0.npy'

# Selection function
print('APOGEE data release is: '+apogee_dr+', and results version is: '+apogee_results_vers)
if os.path.exists(apogee_SF_filename):
    print('Loading APOGEE selection function from '+apogee_SF_filename)
    with open(apogee_SF_filename, 'rb') as f:
        apogee_SF = pickle.load(f)
else:
    sys.exit('Could not find APOGEE selection function, make it. Exiting...')

# Load the effective selection function grid
if os.path.exists(apogee_effSF_filename):
    print('\nLoading effective selection function from '+apogee_effSF_filename)
    with open(apogee_effSF_filename,'rb') as f:
        apogee_effSF_grid_inclArea = pickle.load(f)
else:
    sys.exit('\nFailed to load effective survey section function grid')
    
# Load the effecive selection function mask
if os.path.exists(apogee_effSF_mask_filename):
    print('\nLoading effective selection function mask from '+apogee_effSF_mask_filename)
    apogee_effSF_mask = np.load(apogee_effSF_mask_filename)
else:
    sys.exit('\nFailed to load effective survey section function grid mask')

In [ ]:
# Distance modulus grid
dmods,ds = putil.make_dmod_grid(ndmod,dmod_min,dmod_max)

# Grid of positions in the APOGEE effective selection function grid
Rgrid,phigrid,zgrid = pmass.Rphizgrid(apogee_SF,dmods)

# Apply the effective selection function grid mask
apof_mask = apogee_effSF_grid_inclArea[apogee_effSF_mask]
Rgrid_mask = Rgrid[apogee_effSF_mask]
phigrid_mask = phigrid[apogee_effSF_mask]
zgrid_mask = zgrid[apogee_effSF_mask]

# Include the distance factor in the selection function
apof_mask *= ds**3.*np.log(10)/5.*(dmods[1]-dmods[0])*(np.pi/180.)**2.
#ds_for_delta = 10**(np.append(dmods-(dmods[1]-dmods[0]),dmods[-1]+(dmods[1]-dmods[0]))/5.-2.)
#apof = apof * ds**2. * np.diff(ds_for_delta)

### Load data, begin with a fit that doesn't work
These samples should have $\alpha=2.5$

In [ ]:
# Load data
mock_number = '20'
allstar = np.load('../../data/mock_'+mock_number+'/allstar.npy')
with open('../../data/mock_'+mock_number+'/orbs.pkl','rb') as f:
    orbs = pickle.load(f)
##wi
omask = np.load('../../data/mock_'+mock_number+'/omask.npy')

allstar_mask = allstar[omask]
orbs_mask = orbs[omask]

data_R = orbs_mask.R().value
data_phi = orbs_mask.phi().value
data_z = orbs_mask.z().value

In [ ]:
n_sample = 100
model = pdens.spherical
alpha_range = np.linspace(0.05,5.,num=n_sample)
likelihood = np.zeros_like(alpha_range)

for i in range(len(alpha_range)):
    likelihood[i] = pmass.mloglike([alpha_range[i]], model, apof_mask,
                                   Rgrid_mask, phigrid_mask, zgrid_mask, 
                                   data_R, data_phi, data_z)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(alpha_range, likelihood)
ax.axvline(alpha_range[np.argmin(likelihood)], linestyle='dashed')
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel(r'$\log \mathcal{L}$')

fig.show()

So the fit doesn't work, try and figure out why this is. Plot the on-sky distribution of sources

In [ ]:
# Field info
apogee_field = apogee_SF._apogeeField
n_field = len(apogee_field)
field_glon = apogee_field['GLON']
field_glat = apogee_field['GLAT']
field_locid = apogee_field['LOCATION_ID']

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111)

plot_glon = copy.deepcopy(field_glon)
plot_glat = copy.deepcopy(field_glat)
plot_glon[plot_glon > 180] = plot_glon[plot_glon > 180] - 360

orbs_ll = orbs.ll().value
orbs_bb = orbs.bb().value
orbs_ll[orbs_ll > 180] = orbs_ll[orbs_ll > 180] - 360

orbs_mask_ll = orbs_mask.ll().value
orbs_mask_bb = orbs_mask.bb().value
orbs_mask_ll[orbs_mask_ll > 180] = orbs_mask_ll[orbs_mask_ll > 180] - 360

no_effSF = np.sum(apogee_effSF_grid_inclArea == 0., axis=1) == ndmod

ax.scatter(plot_glon[apogee_effSF_mask], plot_glat[apogee_effSF_mask], 
           c='ForestGreen', s=12, zorder=2)
ax.scatter(plot_glon[~apogee_effSF_mask], plot_glat[~apogee_effSF_mask], 
           c='Red', s=20, zorder=1)
# ax.scatter(orbs_ll, orbs_bb, s=1, color='Black', zorder=3)
ax.scatter(orbs_mask_ll, orbs_mask_bb, s=1, color='Black', zorder=3)

ax.set_xlabel(r'$\ell$ [deg]')
ax.set_ylabel(r'$b$ [deg]')
ax.set_xlim(-180,180)
ax.set_ylim(-90,90)
ax.invert_xaxis()

fig.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

spherrgrid_mask = (Rgrid_mask**2. + zgrid_mask**2.)**0.5

ax.hist(spherrgrid_mask.flatten(), range=(0,10), bins=10)
ax.set_xlabel(r'$r$ of eff. sel. grid bins [kpc]')
ax.set_ylabel('N')
fig.show()

### Now try a fit that does work for comparison
Should be alpha=2.5

In [ ]:
mock_rmin = [2.0,1.0,0.5,0.1]
mock_numbers = ['20','21','22','23']
mock_colors = ['Red','ForestGreen','DodgerBlue','Purple']

fig = plt.figure(figsize=(10,8))
axs = fig.subplots(nrows=2,ncols=2)

for i in range(len(mock_numbers)):
    
    # if i > 2: continue
    
    # Data
    with open('../../data/mock_'+mock_numbers[i]+'/orbs.pkl','rb') as f:
        orbs = pickle.load(f)
    ##wi
    omask = np.load('../../data/mock_'+mock_numbers[i]+'/omask.npy')
    
    # Data is not masked
    rs = orbs.r(use_physical=True).value
    rs_hist,rs_edges = np.histogram(rs, bins=10, range=(0,20))
    rs_cents = (rs_edges[:-1]+rs_edges[1:])/2
    rs_hist_norm = rs_hist / np.max(rs_hist)
    axs[0,0].plot(rs_cents, rs_hist_norm, color=mock_colors[i],
               label='rmin = '+str(mock_rmin[i]))
    if i == 0:
        rs_hist_2 = rs_hist
    else:
        axs[0,1].plot(rs_cents, rs_hist_2-rs_hist, 
                         color=mock_colors[i])
    
    # Data is masked
    orbs_mask = orbs[omask]
    rs = orbs_mask.r(use_physical=True).value
    rs_hist,rs_edges = np.histogram(rs, bins=10, range=(0,20))
    rs_cents = (rs_edges[:-1]+rs_edges[1:])/2
    rs_hist_norm = rs_hist / np.max(rs_hist)
    axs[1,0].plot(rs_cents, rs_hist_norm, color=mock_colors[i],
               label='rmin = '+str(mock_rmin[i]))
    if i == 0:
        rs_hist_mask_2 = rs_hist
    else:
        axs[1,1].plot(rs_cents, rs_hist_mask_2-rs_hist, 
                         color=mock_colors[i])

axs[0,0].annotate('unmasked', xy=(0.1,0.9), xycoords='axes fraction')
axs[0,0].set_xlabel(r'r [kpc]', fontsize=16)
axs[0,0].set_ylabel(r'N normalized', fontsize=16)
axs[0,0].set_xlim(0,20)
axs[0,0].legend()

axs[0,1].set_xlabel(r'r [kpc]', fontsize=16)
axs[0,1].set_ylabel(r'N$_{2.0}$-N', fontsize=16)
axs[0,1].axhline(0, linestyle='dashed', color='Black')
axs[0,1].set_xlim(0,20)

axs[1,0].annotate('masked', xy=(0.1,0.9), xycoords='axes fraction')
axs[1,0].set_xlabel(r'r [kpc]', fontsize=16)
axs[1,0].set_ylabel(r'N normalized', fontsize=16)
axs[1,0].set_xlim(0,20)

axs[1,1].set_xlabel(r'r [kpc]', fontsize=16)
axs[1,1].set_ylabel(r'N$_{2.0}$-N', fontsize=16)
axs[1,1].axhline(0, linestyle='dashed', color='Black')
axs[1,1].set_xlim(0,20)

fig.subplots_adjust(wspace=0.3)
fig.show()

In [ ]:
mock_rmin = [2.0,1.5,1.0,0.5,0.1]
mock_numbers = ['24','25','26','27','28']
mock_colors = ['Red','DarkOrange','ForestGreen','DodgerBlue','Purple']

fig = plt.figure(figsize=(10,8))
axs = fig.subplots(nrows=2,ncols=2)

for i in range(len(mock_numbers)):
    
    # if i > 3: continue
    
    # Data
    with open('../../data/mock_'+mock_numbers[i]+'/orbs.pkl','rb') as f:
        orbs = pickle.load(f)
    ##wi
    omask = np.load('../../data/mock_'+mock_numbers[i]+'/omask.npy')
    
    # Data is not masked
    rs = orbs.r(use_physical=True).value
    rs_hist,rs_edges = np.histogram(rs, bins=10, range=(0,20))
    rs_cents = (rs_edges[:-1]+rs_edges[1:])/2
    rs_hist_norm = rs_hist / np.max(rs_hist)
    axs[0,0].plot(rs_cents, rs_hist_norm, color=mock_colors[i],
               label='rmin = '+str(mock_rmin[i]))
    if i == 0:
        rs_hist_2 = rs_hist
    else:
        axs[0,1].plot(rs_cents, rs_hist_2-rs_hist, 
                         color=mock_colors[i])
    
    # Data is masked
    orbs_mask = orbs[omask]
    rs = orbs_mask.r(use_physical=True).value
    rs_hist,rs_edges = np.histogram(rs, bins=10, range=(0,20))
    rs_cents = (rs_edges[:-1]+rs_edges[1:])/2
    rs_hist_norm = rs_hist / np.max(rs_hist)
    axs[1,0].plot(rs_cents, rs_hist_norm, color=mock_colors[i],
               label='rmin = '+str(mock_rmin[i]))
    if i == 0:
        rs_hist_mask_2 = rs_hist
    else:
        axs[1,1].plot(rs_cents, rs_hist_mask_2-rs_hist, 
                         color=mock_colors[i])

axs[0,0].annotate('unmasked', xy=(0.1,0.9), xycoords='axes fraction')
axs[0,0].set_xlabel(r'r [kpc]', fontsize=16)
axs[0,0].set_ylabel(r'N normalized', fontsize=16)
axs[0,0].set_xlim(0,20)
axs[0,0].legend()

axs[0,1].set_xlabel(r'r [kpc]', fontsize=16)
axs[0,1].set_ylabel(r'N$_{2.0}$-N', fontsize=16)
axs[0,1].axhline(0, linestyle='dashed', color='Black')
axs[0,1].set_xlim(0,20)

axs[1,0].annotate('masked', xy=(0.1,0.9), xycoords='axes fraction')
axs[1,0].set_xlabel(r'r [kpc]', fontsize=16)
axs[1,0].set_ylabel(r'N normalized', fontsize=16)
axs[1,0].set_xlim(0,20)

axs[1,1].set_xlabel(r'r [kpc]', fontsize=16)
axs[1,1].set_ylabel(r'N$_{2.0}$-N', fontsize=16)
axs[1,1].axhline(0, linestyle='dashed', color='Black')
axs[1,1].set_xlim(0,20)

fig.subplots_adjust(wspace=0.3)
fig.show()

In [ ]:
mock_rmin = [2.0,1.5,1.0,0.5,0.1]
mock_numbers = ['29','30','31','32','33']
mock_colors = ['Red','DarkOrange','ForestGreen','DodgerBlue','Purple']

fig = plt.figure(figsize=(10,8))
axs = fig.subplots(nrows=2,ncols=2)

for i in range(len(mock_numbers)):
    
    # if i > 3: continue
    
    # Data
    with open('../../data/mock_'+mock_numbers[i]+'/orbs.pkl','rb') as f:
        orbs = pickle.load(f)
    ##wi
    omask = np.load('../../data/mock_'+mock_numbers[i]+'/omask.npy')
    
    # Data is not masked
    rs = orbs.r(use_physical=True).value
    rs_hist,rs_edges = np.histogram(rs, bins=10, range=(0,20))
    rs_cents = (rs_edges[:-1]+rs_edges[1:])/2
    rs_hist_norm = rs_hist / np.max(rs_hist)
    axs[0,0].plot(rs_cents, rs_hist_norm, color=mock_colors[i],
               label='rmin = '+str(mock_rmin[i]))
    if i == 0:
        rs_hist_2 = rs_hist
    else:
        axs[0,1].plot(rs_cents, rs_hist_2-rs_hist, 
                         color=mock_colors[i])
    
    # Data is masked
    orbs_mask = orbs[omask]
    rs = orbs_mask.r(use_physical=True).value
    rs_hist,rs_edges = np.histogram(rs, bins=10, range=(0,20))
    rs_cents = (rs_edges[:-1]+rs_edges[1:])/2
    rs_hist_norm = rs_hist / np.max(rs_hist)
    axs[1,0].plot(rs_cents, rs_hist_norm, color=mock_colors[i],
               label='rmin = '+str(mock_rmin[i]))
    if i == 0:
        rs_hist_mask_2 = rs_hist
    else:
        axs[1,1].plot(rs_cents, rs_hist_mask_2-rs_hist, 
                         color=mock_colors[i])

axs[0,0].annotate('unmasked', xy=(0.1,0.9), xycoords='axes fraction')
axs[0,0].set_xlabel(r'r [kpc]', fontsize=16)
axs[0,0].set_ylabel(r'N normalized', fontsize=16)
axs[0,0].set_xlim(0,20)
axs[0,0].legend()

axs[0,1].set_xlabel(r'r [kpc]', fontsize=16)
axs[0,1].set_ylabel(r'N$_{2.0}$-N', fontsize=16)
axs[0,1].axhline(0, linestyle='dashed', color='Black')
axs[0,1].set_xlim(0,20)

axs[1,0].annotate('masked', xy=(0.1,0.9), xycoords='axes fraction')
axs[1,0].set_xlabel(r'r [kpc]', fontsize=16)
axs[1,0].set_ylabel(r'N normalized', fontsize=16)
axs[1,0].set_xlim(0,20)

axs[1,1].set_xlabel(r'r [kpc]', fontsize=16)
axs[1,1].set_ylabel(r'N$_{2.0}$-N', fontsize=16)
axs[1,1].axhline(0, linestyle='dashed', color='Black')
axs[1,1].set_xlim(0,20)

fig.subplots_adjust(wspace=0.3)
fig.show()

### Also examine how close the fields are to one another

In [ ]:
from astropy.coordinates import SkyCoord

In [ ]:
# Field info
apogee_field = apogee_SF._apogeeField
n_field = len(apogee_field)
field_glon = apogee_field['GLON']
field_glat = apogee_field['GLAT']
field_locid = apogee_field['LOCATION_ID']

In [ ]:
apo_sc = SkyCoord(l=field_glon*apu.deg, b=field_glat*apu.deg, frame='galactic')
idx,sep,_ = apo_sc.match_to_catalog_sky(apo_sc, nthneighbor=2)
sep = sep.value

In [ ]:
rads_mask = np.array([apogee_SF.radius(locid) for locid in field_locid[apogee_effSF_mask]])
sep_mask = sep[apogee_effSF_mask]

In [ ]:
low_sep_mask = sep_mask < rads_mask

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111)

plot_glon = copy.deepcopy(field_glon[apogee_effSF_mask])
plot_glat = copy.deepcopy(field_glat[apogee_effSF_mask])
plot_glon[plot_glon > 180] = plot_glon[plot_glon > 180] - 360

ax.scatter(plot_glon[~low_sep_mask], plot_glat[~low_sep_mask], 
           c='ForestGreen', s=12, zorder=2)
ax.scatter(plot_glon[low_sep_mask], plot_glat[low_sep_mask], 
           c='Red', s=20, zorder=1)
# ax.scatter(orbs_mask_ll, orbs_mask_bb, s=1, color='Black', zorder=3)

ax.set_xlabel(r'$\ell$ [deg]')
ax.set_ylabel(r'$b$ [deg]')
ax.set_xlim(-180,180)
ax.set_ylim(-90,90)
ax.invert_xaxis()

fig.show()

Maybe cut out these fields and re-make some mocks